In [1]:
import os
print(os.getcwd())
print('files')
print(os.listdir())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/ci-ex1/code/Users/odl_user_226850
files
['.amlignore', '.amlignore.amltmp', '.ipynb_aml_checkpoints', 'outputs', 'train.py', 'udacity-project (1) (6).ipynb', 'udacity-project (1) (6).ipynb.amltmp']


In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-226850
Azure region: westeurope
Subscription id: 9a7511b8-150f-4a58-8528-3e7d50216c31
Resource group: aml-quickstarts-226850


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "proj1cluster"

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###

# Verify that the cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [5]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({'--C': uniform(0.1, 1.0), '--max_iter': choice(50,100,200)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

    
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory=".",
                      script='train.py',
                      compute_target=cpu_cluster,
                      environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                              hyperparameter_sampling=ps,
                              policy=policy,
                              primary_metric_name="accuracy",
                              primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                              max_total_runs=100,
                              max_concurrent_runs=4)

In [6]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
from azureml.widgets import RunDetails

hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e
Web View: https://ml.azure.com/runs/HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e?wsid=/subscriptions/9a7511b8-150f-4a58-8528-3e7d50216c31/resourcegroups/aml-quickstarts-226850/workspaces/quick-starts-ws-226850&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-02-27T11:14:45.320345][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-02-27T11:14:46.3155453Z][SCHEDULER][INFO]Scheduling job, id='HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_0' 
[2023-02-27T11:14:46.3659082Z][SCHEDULER][INFO]Scheduling job, id='HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_1' 
[2023-02-27T11:14:46.5749667Z][SCHEDULER][INFO]Scheduling job, id='HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_2' 
[2023-02-27T11:14:46.8013889Z][SCHEDULER][INFO]Scheduling job, id='HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_3' 
[2023-02-27T11:14:46.533242][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e',
 'target': 'proj1cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-27T11:14:44.400345Z',
 'endTimeUtc': '2023-02-27T12:02:26.72667Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'f64c560d-baf3-41dd-ae11-52c7917ce438',
  'user_agent': 'python/3.8.10 (Linux-5.15.0-1031-azure-x86_64-with-glibc2.17) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.48.0',
  'space_size': 'infinite_space_size',
  'score': '0.9154274152756702',
  'best_child_run_id': 'HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_26',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_c1d7557a-02ca-4ab9-b299-9f45cd8f794e_26'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryV

KeyError: 'log_files'

In [7]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
final_model = best_run.register_model(model_name = 'hyp_best_run',model_path = 'outputs/model.joblib')

In [8]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###
file_url= "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=file_url)

In [9]:
from train import clean_data
import pandas as pd
from azureml.core import Dataset

# Use the clean_data function to clean your data.
X, y = clean_data(ds)
y=pd.DataFrame(y)
df_full=pd.concat([X,y],axis=1)

from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df_full,test_size=0.3)
train_data.head()

train_data.to_csv('training_data.csv')

ds1= ws.get_default_datastore()
ds1.upload(src_dir='.', target_path='training_data')

train_ds = Dataset.Tabular.from_delimited_files(path=[(ds1, ('training_data/training_data.csv'))])


"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.


Uploading an estimated of 9 files
Uploading ./.amlignore
Uploaded ./.amlignore, 1 files out of an estimated total of 9
Uploading ./.amlignore.amltmp
Uploaded ./.amlignore.amltmp, 2 files out of an estimated total of 9
Uploading ./train.py
Uploaded ./train.py, 3 files out of an estimated total of 9
Uploading ./conda_dependencies.yml
Uploaded ./conda_dependencies.yml, 4 files out of an estimated total of 9
Uploading ./training_data.csv
Uploaded ./training_data.csv, 5 files out of an estimated total of 9
Uploading ./udacity-project (1) (6).ipynb
Uploaded ./udacity-project (1) (6).ipynb, 6 files out of an estimated total of 9
Uploading ./udacity-project (1) (6).ipynb.amltmp
Uploaded ./udacity-project (1) (6).ipynb.amltmp, 7 files out of an estimated total of 9
Uploading ./.ipynb_aml_checkpoints/udacity-project (1) (6)-checkpoint2023-1-27-11-3-41Z.ipynb
Uploaded ./.ipynb_aml_checkpoints/udacity-project (1) (6)-checkpoint2023-1-27-11-3-41Z.ipynb, 8 files out of an estimated total of 9
Upload

In [10]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_ds,
    label_column_name='y',
    n_cross_validations=5,
    compute_target = cpu_cluster)

In [11]:
# Submit your automl run

### YOUR CODE HERE ###
automl_run= exp.submit(automl_config, show_output = True)

Submitting remote run.
No run_configuration provided, running on proj1cluster with default configuration
Running on remote compute: proj1cluster


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_f8fbdcac-72e3-4186-8108-e92c3ad0652e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

In [13]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###
#best_run
best_run, fitted_model = automl_run.get_output()
print(best_run)
print(fitted_model)

import joblib
joblib.dump(fitted_model,'automl_best_model.pkl')

from azureml.core.model import Model

model = Model.register(workspace=ws, model_path="automl_best_model.pkl", model_name="automl-best-model")

Package:azureml-automl-runtime, training version:1.48.0.post1, current version:1.47.0
Package:azureml-core, training version:1.48.0, current version:1.47.0
Package:azureml-dataprep, training version:4.8.3, current version:4.5.7
Package:azureml-dataprep-rslex, training version:2.15.1, current version:2.11.4
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.47.0
Package:azureml-defaults, training version:1.48.0, current version:1.47.0
Package:azureml-interpret, training version:1.48.0, current version:1.47.0
Package:azureml-mlflow, training version:1.48.0, current version:1.47.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.47.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.47.0
Package:azureml-telemetry, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.47.0
Package:azureml-train-automl-runtime, training version:1.48.0, current v

Run(Experiment: udacity-project,
Id: AutoML_bf69886b-7d9a-4c1c-a6a3-e480e07abffe_30,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('0', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('lightgbmclassifier', LightGBMClassifier(min_data_in_leaf=20, n_jobs=1, problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 'cpu'}), random_state=None))], verbose=False)), ('26', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False